In [7]:
SEASON = '1988-89'

In [8]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df['PlayerScore'] = p_df['PlayerScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Michael Jordan,CHI,SG,133.480767,25,81,81,40.2,11.9,22.2,...,32.1,14.6,5.2,19.8,0.292,8.4,3.4,11.9,11.4,47.0
2,Magic Johnson,LAL,PG,104.570923,29,77,77,37.5,7.5,14.8,...,24.3,12.1,4.0,16.1,0.267,7.3,2.0,9.4,8.3,57.0
3,Charles Barkley,PHI,PF,90.122182,25,79,79,39.1,8.9,15.3,...,24.4,13.2,2.9,16.1,0.250,7.3,0.2,7.5,7.4,46.0
4,John Stockton,UTA,PG,89.063197,26,82,82,38.7,6.1,11.3,...,19.1,9.8,5.8,15.6,0.236,5.1,3.2,8.3,8.3,51.0
5,Karl Malone,UTA,PF,81.152782,25,80,80,39.1,10.1,19.5,...,30.4,8.6,6.6,15.2,0.233,4.3,1.2,5.4,5.9,51.0
6,Clyde Drexler,POR,SG,74.335370,26,78,78,39.3,10.6,21.4,...,27.2,8.8,3.8,12.6,0.197,5.4,1.1,6.5,6.6,39.0
7,Hakeem Olajuwon,HOU,C,73.382129,26,82,82,36.9,9.6,19.0,...,28.2,4.6,7.8,12.4,0.197,2.7,2.9,5.5,5.7,45.0
8,Kevin Johnson,PHO,PG,65.817441,22,81,81,39.2,7.0,13.9,...,21.2,8.6,3.5,12.2,0.184,3.9,0.3,4.2,5.0,55.0
9,Patrick Ewing,NYK,C,62.648397,26,80,80,36.2,9.1,16.0,...,23.6,6.2,4.7,10.9,0.181,2.3,2.3,4.5,4.8,52.0
10,Fat Lever,DEN,SG,58.661179,28,71,71,38.7,7.9,17.2,...,21.3,5.1,4.2,9.2,0.161,3.4,1.8,5.2,5.0,44.0


In [9]:
t_df

,Team,OverallW,OverallL,HomeW,HomeL,RoadW,RoadL,EastW,EastL,WestW,...,DecL,JanW,JanL,FebW,FebL,MarW,MarL,AprW,AprL,TeamCode
0,Atlanta Hawks,52,30,33,8,19,22,38,18,14,...,3,7,7,9,4,7,8,10,2,ATL
1,Boston Celtics,42,40,32,9,10,31,27,29,15,...,7,7,8,6,7,11,4,5,7,BOS
2,Charlotte Hornets,20,62,12,29,8,33,12,44,8,...,9,3,13,4,8,2,13,3,9,CHO
3,Chicago Bulls,47,35,30,11,17,24,28,28,19,...,4,9,5,9,4,10,6,4,8,CHI
4,Cleveland Cavaliers,57,25,37,4,20,21,38,18,19,...,2,11,4,10,3,9,8,6,5,CLE
5,Dallas Mavericks,38,44,24,17,14,27,8,16,30,...,5,4,11,8,5,3,14,6,5,DAL
6,Denver Nuggets,44,38,35,6,9,32,12,12,32,...,7,5,11,8,4,8,6,6,6,DEN
7,Detroit Pistons,63,19,37,4,26,15,41,15,22,...,4,8,6,8,3,16,1,11,2,DET
8,Golden State Warriors,43,39,29,12,14,27,14,10,29,...,7,11,5,9,2,9,10,3,8,GSW
9,Houston Rockets,45,37,31,10,14,27,12,12,33,...,5,7,6,6,6,8,8,6,6,HOU
